In [5]:
%matplotlib inline

In [6]:
import pickle
import pandas as pd
import numpy as np

In [7]:
import flatten as fat

In [8]:
DATA_DIR = './utah_data/'

In [9]:
climb = fat.combine_pickle(DATA_DIR)

In [10]:
climb.shape

(11852, 43)

In [14]:
# isolate grade columns
import re
rate = [col for col in climb.columns if re.search(r'^rate', col)]
grade = climb[rate]
grade.shape

(11852, 8)

In [15]:
#grade = grade.dropna(axis=0, how='all')

In [16]:
import numpy as np
def convert_hueco(hueco, LEAST_DIFFICULT=float(0), FIRST_STEP=float(1)):

    if not pd.isnull(hueco):
        hueco = hueco[1:]

        if hueco == '-easy':
            # lower bound
            hard = LEAST_DIFFICULT
        else:
            # V0 will be equal to this
            hard = LEAST_DIFFICULT + FIRST_STEP

            # deal with the numbers in the string
            if re.search(r'(\+$)|(-$)', hueco):
                pnt = float(hueco[:-1].strip())
                # add/take a half a point off for plus minus
                if hueco[-1] == '+':
                    hard += pnt + .5
                else:
                    hard += pnt - .5
            elif re.search(r'(\d+)[- ]+(\d+)', hueco):
                # return the mean of a range
                lower = re.findall(r'\d+', hueco)[0].strip()
                upper = re.findall(r'\d+', hueco)[1].strip()
                hard += np.mean( [float(lower), float(upper)] )
            elif re.search(r'\d+', hueco):                    
                try:
                    # cast as float ignoring sign
                    core_rate = float(hueco.strip('+-'))
                except:
                    hard = None
                else:
                    hard += core_rate

        return hard

In [17]:
convert_hueco('V45-46')

46.5

In [18]:
climb['floatHueco'] = map(convert_hueco, climb['rateHueco'])

In [19]:
np.median(climb['floatHueco'].dropna())

4.0

In [145]:
def convert_ZA(ZA):
    if not pd.isnull(ZA):
        return float(ZA)
# convert_lead(climb['floatZA'])

In [146]:
from statsmodels.distributions.empirical_distribution import ECDF
def scale01(feature):
    # use on an entire column
    ecdf = ECDF(feature.dropna())
    qtile = ecdf(feature)
    qtile[qtile == 1.0] = float('NaN')
    return qtile

In [147]:
climb['pctHueco'] = scale01(climb['floatHueco'])
climb['pctZA'] = scale01(climb['floatZA'])

In [148]:
climb['gradeComb'] = climb[['pctHueco','pctZA']].mean(axis='columns')

In [149]:
# climb['gradeComb'].hist()

In [150]:
# IDEAL IS A BETA DISTRIBUTION FOR PRIOR

In [151]:
def score_climb_grade(grade, ideal_grade):
    ABOVE_DECAY = 5
    BELOW_DECAY = 2
    if grade == ideal_grade:
        return 1.0
    elif grade > ideal_grade:
        diff = grade - ideal_grade
        return (1-diff) ** ABOVE_DECAY
    else:
        diff = ideal_grade - grade
        return (1-diff) ** BELOW_DECAY

# grade = np.array([.5,.7,.3,.6,.9])

In [165]:
sample = climb[:10]


/v/beaver-boulder/108131427                          NaN
/v/spider-man-/108131436                        0.421247
/v/burr-trail--long-canyon/105973758                 NaN
/v/58-corner/106448362                          0.493234
/v/bastard-child/105973780                      0.509457
/v/beam-bump/105973810                          0.304860
/v/flipper-babies/105973765                     0.493234
/v/longs-lies/105973794                         0.693831
/v/capitol-reef-national-park/105716802              NaN
/v/basketball-wallslickrock-divide/105968325         NaN
Name: gradeComb, dtype: float64

In [142]:
from time import time

In [ ]:
def grade_similarity(climb):

In [172]:
def grade_similarity(climb):
    
    collect = []
    for ideal_grade in climb['gradeComb']:
        scores = [score_climb_grade(g, ideal_grade) for g in climb['gradeComb']]
        recco = pd.DataFrame({'recco':scores}, index=climb.index)
        collect.append(recco)
    grade_matrix = pd.concat(collect, axis='columns')
    grade_matrix.columns = climb.index
    return grade_matrix

In [173]:
print 'Converting grades to float...'
t0 = time()
climb['floatHueco'] = map(convert_hueco, climb['rateHueco'])
climb['pctHueco'] = scale01(climb['floatHueco'])

climb['floatZA'] = map(convert_ZA, climb['rateZA'])
climb['pctZA'] = scale01(climb['floatZA'])

climb['gradeComb'] = climb[['pctHueco','pctZA']].mean(axis='columns')
print 'took %0.2f seconds' % (time() - t0)

print 'Generating grade similarity matrix...'
t0 = time()
grade_matrix = grade_similarity(climb)
print 'took %d seconds' % (time() - t0)

Generating grade similarity matrix
Cross Grading 1000 records took 1 seconds


In [175]:
grade_matrix.to_pickle('_grade_matrix')
x = pd.read_pickle('_grade_matrix')

print "X is", x.shape, type(x)

X is (1000, 1000) <class 'pandas.core.frame.DataFrame'>
